In [1]:
def binary_classification_exec(epoch_count = 10, mb_size = 10, report=1, train_ratio = 0.6, val_ratio = 0.2):
    binary_load_dataset()
    init_param()
    train_metrics_mean_row, val_metrics_row, test_metrics = train_and_test(epoch_count, 
                                                                                mb_size, 
                                                                                report, 
                                                                                train_ratio,
                                                                                val_ratio)
    return train_metrics_mean_row, val_metrics_row, test_metrics

In [2]:
def binary_load_dataset():
    # 신경망 설계할 때는 mini dataset 으로 진행합니다.
    #with open('/content/pulsar_stars_mini.csv') as csvfile:
    with open('/content/pulsar_stars.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)
        rows = []
        for row in csvreader:
            rows.append(row)

    global data, input_cnt, output_cnt

    input_cnt, output_cnt = 8, 1
    data = np.asarray(rows, dtype='float32')

In [3]:
def init_param():
    global weight, bias

    weight = np.random.normal(RND_MEAN, RND_STD, size = [input_cnt, output_cnt])
    bias   = np.zeros([output_cnt])      

    print("Initial weight Value : \n{}".format(weight))
    print("Initial bias Value : \n{}".format(bias))

In [4]:
def arrange_data(mb_size, train_ratio, val_ratio):

    global shuffle_map, test_begin_index, val_begin_index

    shuffle_map = np.arange(data.shape[0])

    np.random.shuffle(shuffle_map)
    mini_batch_step_count = int(data.shape[0] * train_ratio) // mb_size

    val_begin_index  = mini_batch_step_count * mb_size
    test_begin_index = int(val_begin_index + (val_ratio * data.shape[0]))
    
    return mini_batch_step_count    

In [5]:
def get_test_data():
    test_data = data[shuffle_map[test_begin_index : ]]
    return test_data[:,:-output_cnt], test_data[:,-output_cnt:]

def get_val_data():
    val_data = data[shuffle_map[val_begin_index : test_begin_index]]
    return val_data[:,:-output_cnt], val_data[:,-output_cnt:]

def get_train_data(mb_size, n):
    from_idx = n * mb_size
    to_idx   = (n + 1) * mb_size

    train_data = data[shuffle_map[from_idx : to_idx]]
    
    return train_data[:,:-output_cnt], train_data[:,-output_cnt:]

In [ ]:
def train_and_test(epoch_count, mb_size, report, train_ratio, val_ratio):

    mini_batch_step_count = arrange_data(mb_size,train_ratio, val_ratio)

    test_x, test_y = get_test_data()
    val_x,  val_y  = get_val_data()

    losses_mean_row = []
    val_loss_row    = []

    #losses_mean_row, accs_mean_row = [], []
    #val_loss_row, val_acc_row      = [], []
    
    for epoch in range(epoch_count):

        losses = []
        #accs = []

        for n in range(mini_batch_step_count):
            train_x, train_y  = get_train_data(mb_size, n)
           
            loss, _           = run_train(train_x,train_y)
               
            losses.append(loss)
            #accs.append(acc)

        val_loss, val_acc = run_test(val_x, val_y)
        val_loss_row.append(val_loss)
        #val_acc_row.append(val_acc)  

        if report > 0 and (epoch+1) % report == 0:
            
            print("Epoch {} : Train - Loss = {:.3f} / Val - Loss = {:.3f}, Acc = {:.3f}, Precision = {:.3f}, Recall = {:.3f}, F1 = {:.3F}".\
                  format(epoch+1, np.mean(losses), val_loss, val_acc[0], val_acc[1], val_acc[2], val_acc[3]))
            
        losses_mean = np.mean(losses) 
        #accs_mean = np.mean(accs)

        losses_mean_row.append(losses_mean)  
        #accs_mean_row.append(accs_mean)   



    test_loss, test_acc = run_test(test_x, test_y)
    
    print("\n","=" * 50, 'Final Test', '=' * 50)
    print('\nTest Acc = {:.3f}, Precision = {:.3f}, Recall = {:.3f}, F1 = {:.3F}'.\
          format(test_acc[0], test_acc[1], test_acc[2], test_acc[3]))
    print('\nLoss = {:.3f}'.format(test_loss))

    #return [losses_mean_row, accs_mean_row], [val_loss_row, val_acc_row], [test_loss , test_acc]
    return [losses_mean_row], [val_loss_row], [test_loss , test_acc]

In [ ]:
def forward_neuralnet(x):
    y_hat = np.matmul(x, weight) + bias
    return y_hat, x

In [6]:
def relu(x):
    return np.maximum(x,0)

In [ ]:
def sigmoid_cross_entropy_with_logits(z,x):
    return relu(x) - x * z + np.log(1+np.exp(-np.abs(x)))

In [ ]:
def forward_postproc(output, y):
    CEE  = sigmoid_cross_entropy_with_logits(y,output)
    loss = np.mean(CEE)
    
    return loss, [y, output, CEE]

In [ ]:
def backprop_neuralnet(G_output, x):
    global weight, bias

    x_transpose = x.transpose()
    G_w = np.matmul(x_transpose, G_output)

    G_b = np.sum(G_output, axis = 0)

    weight -= LEARNING_RATE * G_w
    bias   -= LEARNING_RATE * G_b

In [ ]:
def sigmoid_cross_entropy_with_logits_derv(z, x):
    return -z + sigmoid(x)

In [ ]:
def sigmoid(x):
    return np.exp(-relu(-x)) / (1.0 + np.exp(-np.abs(x)))

In [ ]:
def backprop_postproc():
    
    g_loss_entropy = 1.0/np.prod(CEE.shape)
    g_entropy_output = sigmoid_cross_entropy_with_logits_derv(y,output)
    
    G_output = g_entropy_output * g_loss_entropy 
    
    return G_output

In [ ]:
def backprop_postproc(aux_pp_y_output_CEE):
    
    y, output, CEE = aux_pp_y_output_CEE

    g_loss_entropy = 1.0/np.prod(CEE.shape)
    g_entropy_output = sigmoid_cross_entropy_with_logits_derv(y,output)
    
    G_output = g_entropy_output * g_loss_entropy 
    
    return G_output

In [ ]:
def eval_accuracy_numpy(output,y):

    est_yes = np.greater(output,0)
    ans_yes = np.greater(y, 0.5)

    est_no = np.logical_not(est_yes) 
    ans_no = np.logical_not(ans_yes)
  
    tp = np.sum(np.logical_and(est_yes, ans_yes))
    tn = np.sum(np.logical_and(est_no, ans_no))
    fp = np.sum(np.logical_and(ans_no, est_yes))
    fn = np.sum(np.logical_and(ans_yes, est_no))

    accuracy = safe_div(tp+tn,tp+fp+fn+tn)
    precision = safe_div(tp,tp+fp)
    recall = safe_div(tp,tp+fn)
    f1 = 2 * safe_div(recall*precision,recall+precision)
    
    return [accuracy, precision, recall, f1]

In [ ]:
def run_train(x, y):
    y_hat, aux_nn_x           = forward_neuralnet(x)
    loss, aux_pp_y_output_CEE = forward_postproc(y_hat, y)

    accuracy = eval_accuracy_numpy(y_hat, y)
    
    G_output = backprop_postproc(aux_pp_y_output_CEE)
    backprop_neuralnet(G_output, aux_nn_x)
    
    return loss, accuracy

def run_test(x, y):
    y_hat, _ = forward_neuralnet(x)
    loss,  _ = forward_postproc(y_hat, y)
    accuracy = eval_accuracy_numpy(y_hat, y)
    return loss, accuracy

In [ ]:
def safe_div(p, q):
    p, q = float(p), float(q)
    if np.abs(q) < 1.0e-20:
        return np.sign(p)
    return p / q